OpenAI Generation

In [ ]:
# Install the newer OpenAI SDK
!pip install --upgrade openai

from openai import OpenAI
import pandas as pd
import random
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.8.1
    Uninstalling openai-2.8.1:
      Successfully uninstalled openai-2.8.1


In [ ]:
client = OpenAI(api_key="KEY_GOES_HERE")

def query_chatGPT(prompt):
    """Send a prompt to OpenAI GPT-4o."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",       # you can change to gpt-4o if you want
        messages=[{"role": "user", "content": prompt}],
        max_tokens=400,
        temperature=1
    )
    return response.choices[0].message.content

In [ ]:
# download public domain poems
poems_df = pd.read_csv(
    'https://github.com/maria-antoniak/poetry-eval/'
    'raw/refs/heads/main/data/poetry-evaluation_public-domain-poems.csv'
)

# keep sonnets & ghazals
poems_df = poems_df[poems_df['form'].isin(['sonnet', 'ghazal'])]

# random 25 sonnets + 2 ghazals
sample_df = poems_df[poems_df['form'] == 'sonnet'].sample(25)
sample_df = pd.concat([sample_df, poems_df[poems_df['form'] == 'ghazal']])

# load brown poems
import gdown
url = "https://drive.google.com/file/d/1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz/view?usp=sharing"
output = "brown_poems.csv"
gdown.download(url=url, output=output, fuzzy=True)

brown_df = pd.read_csv("brown_poems.csv")

# reduce sample_df columns
sample_df = sample_df[['author', 'poem_title', 'poem_text', 'form']]

# combine datasets
combined_df = pd.concat([sample_df, brown_df])

Downloading...
From: https://drive.google.com/uc?id=1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz
To: /content/brown_poems.csv
100%|██████████| 3.18k/3.18k [00:00<00:00, 7.24MB/s]


In [ ]:
prompt_template_start = '''
Read the following poem and classify its form as: [sonnet, ghazal, duplex].
You must choose exactly ONE.

Return the answer in this format:

1. Poetic Form: ...
2. Elaborated Rationale: ...
3. One-Word Summary: ...
4. Confidence Score: ...

Poem Text:
'''

prompt_template_end = '''

Pick ONE of these forms: [sonnet, ghazal, duplex].
'''

In [ ]:
def extract_response_fields(response):
    """Parse the classification output."""
    lines = response.strip().split('\n')

    result = {
        'poetic_form': None,
        'rationale': None,
        'summary': None,
        'confidence': None
    }

    for line in lines:
        if line.startswith('1. Poetic Form:'):
            result['poetic_form'] = line.split(':', 1)[1].strip()
        elif line.startswith('2. Elaborated Rationale:'):
            result['rationale'] = line.split(':', 1)[1].strip()
        elif line.startswith('3. One-Word Summary:'):
            result['summary'] = line.split(':', 1)[1].strip()
        elif line.startswith('4. Confidence Score:'):
            result['confidence'] = line.split(':', 1)[1].strip()

    return result

In [ ]:
for index, row in combined_df.iterrows():
    prompt = prompt_template_start + row['poem_text'] + prompt_template_end
    response = query_chatGPT(prompt)
    fields = extract_response_fields(response)

    combined_df.at[index, 'poetic_form'] = fields['poetic_form']
    combined_df.at[index, 'rationale'] = fields['rationale']
    combined_df.at[index, 'summary'] = fields['summary']
    combined_df.at[index, 'confidence'] = fields['confidence']

combined_df.to_csv("combined_df.csv", index=False)

In [ ]:
subjects = [
    "abuse",
    "activities",
    "arts & sciences",
    "christianity",
    "chronic illness",
    "crime & punishment",
    "cycles",
    "desire & passion",
    "doubt & contemplation",
    "family & ancestors",
    "gender & sexuality",
    "greek & roman mythology",
    "history & politics",
    "home life",
    "hope",
    "humor & satire",
    "indoor activities",
    "life choices",
    "lgbtq+",
    "living",
    "love",
    "love death avoidance",
    "memory & nostalgia",
    "men & women",
    "mythology & folklore",
    "nature",
    "power",
    "poetry & poets",
    "race & ethnicity",
    "relationships",
    "romantic love",
    "sexual violence",
    "sorrow & grieving",
    "social commentaries",
    "subjugation",
    "the mind",
    "trees and flowers",
    "truth",
    "twilight",
    "urban environment",
    "war & conflict",
    "weather",
    "youth"
]


def generate_general_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}."


def generate_figurative_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Do not use the words {subject} or {form}."


def generate_specific_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Make it about something specific."


new_poems_df = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])

for subject in subjects:
    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    genResponse = query_chatGPT(genPrompt)
    figResponse = query_chatGPT(figPrompt)
    specResponse = query_chatGPT(specPrompt)

    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "general", genResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "figurative", figResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "specific", specResponse]

new_poems_df.to_csv("new_poems_df.csv", index=False)

print("All tasks complete.")

All tasks complete.


In [ ]:
BROWN_RULES = """
Write a poem in the Duplex form as defined by Jericho Brown.
You MUST follow ALL of these rules:

1. The poem has exactly 14 lines.
2. Each line must contain between 9 and 11 syllables.
3. The poem should blend qualities of ghazal, sonnet, and blues traditions.
4. Line 1 is repeated as Line 14.
5. Line 2 must reinterpret Line 1 in an unexpected way.
6. Line 2 is repeated as Line 3.
7. Line 4 must reinterpret Line 3 in an unexpected way.
8. This pattern of echo / reinterpretation continues until Line 13.
9. Line 13 becomes the first line of the couplet that leads to the final line (Line 14).
10. The poem’s theme should be rooted in emotional tension, reflection, or personal revelation.
"""

In [ ]:
def generate_general_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"The poem MUST follow all Duplex rules above."
    )


def generate_figurative_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Do NOT use the words '{subject}' or '{form}' anywhere in the poem.\n"
        f"The poem MUST follow all Duplex rules above."
    )


def generate_specific_prompt(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Make the poem about a very specific moment, object, place, or memory.\n"
        f"The poem MUST follow all Duplex rules above."
    )

In [ ]:
new_brown_poems_df = pd.DataFrame(
    columns=['subject', 'form', 'prompt_type', 'poem_text']
)

In [ ]:
for subject in subjects:

    # build 3 prompt types
    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    # call your model function (this is unchanged)
    genResponse = query_chatGPT(genPrompt)
    figResponse = query_chatGPT(figPrompt)
    specResponse = query_chatGPT(specPrompt)

    # store in your preferred format
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "general", genResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "figurative", figResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "specific", specResponse
    ]

In [ ]:
new_brown_poems_df.to_csv("new_brown_poems_df.csv", index=False)

print("All Duplex poems generated and saved.")

All Duplex poems generated and saved.


Claude Generation:

In [ ]:
# Install the Anthropic SDK
!pip install anthropic

import anthropic
import pandas as pd
import random
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 10.4 MB/s eta 0:00:00


In [ ]:
# Initialize the Anthropic client
client = anthropic.Anthropic(
    api_key="KEY_GOES_HERE"
)

def query_claude(prompt):
    """Send a prompt to Claude."""
    message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        temperature=1,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [ ]:
poems_df = pd.read_csv(
    'https://github.com/maria-antoniak/poetry-eval/'
    'raw/refs/heads/main/data/poetry-evaluation_public-domain-poems.csv'
)

# Keep sonnets & ghazals
poems_df = poems_df[poems_df['form'].isin(['sonnet', 'ghazal'])]

# Random 25 sonnets + 2 ghazals
sample_df = poems_df[poems_df['form'] == 'sonnet'].sample(25)
sample_df = pd.concat([sample_df, poems_df[poems_df['form'] == 'ghazal']])


In [ ]:
# load brown poems
import gdown
url = "https://drive.google.com/file/d/1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz/view?usp=sharing"
output = "brown_poems.csv"
gdown.download(url=url, output=output, fuzzy=True)

brown_df = pd.read_csv("brown_poems.csv")

# reduce sample_df columns
sample_df = sample_df[['author', 'poem_title', 'poem_text', 'form']]

# combine datasets
combined_df = pd.concat([sample_df, brown_df])

Downloading...
From: https://drive.google.com/uc?id=1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz
To: /content/brown_poems.csv
100%|██████████| 3.18k/3.18k [00:00<00:00, 4.64MB/s]


In [ ]:
prompt_template_start = '''
Read the following poem and classify its form as: [sonnet, ghazal, duplex].
You must choose exactly ONE.

Return the answer in this format:

1. Poetic Form: ...
2. Elaborated Rationale: ...
3. One-Word Summary: ...
4. Confidence Score: ...

Poem Text:
'''

prompt_template_end = '''

Pick ONE of these forms: [sonnet, ghazal, duplex].
'''

In [ ]:
def extract_response_fields(response):
    """Parse the classification output."""
    lines = response.strip().split('\n')

    result = {
        'poetic_form': None,
        'rationale': None,
        'summary': None,
        'confidence': None
    }

    for line in lines:
        if line.startswith('1. Poetic Form:'):
            result['poetic_form'] = line.split(':', 1)[1].strip()
        elif line.startswith('2. Elaborated Rationale:'):
            result['rationale'] = line.split(':', 1)[1].strip()
        elif line.startswith('3. One-Word Summary:'):
            result['summary'] = line.split(':', 1)[1].strip()
        elif line.startswith('4. Confidence Score:'):
            result['confidence'] = line.split(':', 1)[1].strip()

    return result

In [ ]:
# Classify poems using Claude
for index, row in combined_df.iterrows():
    prompt = prompt_template_start + row['poem_text'] + prompt_template_end
    response = query_claude(prompt)
    fields = extract_response_fields(response)

    combined_df.at[index, 'poetic_form'] = fields['poetic_form']
    combined_df.at[index, 'rationale'] = fields['rationale']
    combined_df.at[index, 'summary'] = fields['summary']
    combined_df.at[index, 'confidence'] = fields['confidence']

combined_df.to_csv("combined_df_claude.csv", index=False)

In [ ]:
# Subjects for poem generation
subjects = [
    "abuse", "activities", "arts & sciences", "christianity", "chronic illness",
    "crime & punishment", "cycles", "desire & passion", "doubt & contemplation",
    "family & ancestors", "gender & sexuality", "greek & roman mythology",
    "history & politics", "home life", "hope", "humor & satire",
    "indoor activities", "life choices", "lgbtq+", "living", "love",
    "love death avoidance", "memory & nostalgia", "men & women",
    "mythology & folklore", "nature", "power", "poetry & poets",
    "race & ethnicity", "relationships", "romantic love", "sexual violence",
    "sorrow & grieving", "social commentaries", "subjugation", "the mind",
    "trees and flowers", "truth", "twilight", "urban environment",
    "war & conflict", "weather", "youth"
]

BROWN_RULES = """
Write a poem in the Duplex form as defined by Jericho Brown.
You MUST follow ALL of these rules:

1. The poem has exactly 14 lines.
2. Each line must contain between 9 and 11 syllables.
3. The poem should blend qualities of ghazal, sonnet, and blues traditions.
4. Line 1 is repeated as Line 14.
5. Line 2 must reinterpret Line 1 in an unexpected way.
6. Line 2 is repeated as Line 3.
7. Line 4 must reinterpret Line 3 in an unexpected way.
8. This pattern of echo / reinterpretation continues until Line 13.
9. Line 13 becomes the first line of the couplet that leads to the final line (Line 14).
10. The poem's theme should be rooted in emotional tension, reflection, or personal revelation.
"""

In [ ]:
def generate_general_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}."

def generate_figurative_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Do not use the words {subject} or {form}."

def generate_specific_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}. Make it about something specific."

# Generate simple Duplex poems
new_poems_df = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])

for subject in subjects:
    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    genResponse = query_claude(genPrompt)
    figResponse = query_claude(figPrompt)
    specResponse = query_claude(specPrompt)

    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "general", genResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "figurative", figResponse]
    new_poems_df.loc[len(new_poems_df)] = [subject, "Duplex", "specific", specResponse]

new_poems_df.to_csv("new_poems_df_claude.csv", index=False)

print("Simple Duplex poems generated and saved.")


Simple Duplex poems generated and saved.


In [ ]:
def generate_general_prompt_brown(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"The poem MUST follow all Duplex rules above."
    )

def generate_figurative_prompt_brown(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Do NOT use the words '{subject}' or '{form}' anywhere in the poem.\n"
        f"The poem MUST follow all Duplex rules above."
    )

def generate_specific_prompt_brown(subject, form="Duplex"):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a poem about the subject of {subject} in the form: {form}.\n"
        f"Make the poem about a very specific moment, object, place, or memory.\n"
        f"The poem MUST follow all Duplex rules above."
    )

# Generate Duplex poems with Brown rules
new_brown_poems_df = pd.DataFrame(
    columns=['subject', 'form', 'prompt_type', 'poem_text']
)

for subject in subjects:
    # Build 3 prompt types
    genPrompt = generate_general_prompt_brown(subject)
    figPrompt = generate_figurative_prompt_brown(subject)
    specPrompt = generate_specific_prompt_brown(subject)

    # Call Claude
    genResponse = query_claude(genPrompt)
    figResponse = query_claude(figPrompt)
    specResponse = query_claude(specPrompt)

    # Store results
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "general", genResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "figurative", figResponse
    ]
    new_brown_poems_df.loc[len(new_brown_poems_df)] = [
        subject, "Duplex", "specific", specResponse
    ]

new_brown_poems_df.to_csv("new_brown_poems_df_claude.csv", index=False)

print("All Duplex poems with Brown rules generated and saved.")

All Duplex poems with Brown rules generated and saved.


In [ ]:
print("\n=== All tasks complete using Claude API ===")
print(f"Files created:")
print("  - combined_df_claude.csv")
print("  - new_poems_df_claude.csv")
print("  - new_brown_poems_df_claude.csv")


=== All tasks complete using Claude API ===
Files created:
  - combined_df_claude.csv
  - new_poems_df_claude.csv
  - new_brown_poems_df_claude.csv


Olmo

In [ ]:
!pip install requests
HF_API_KEY = "KEY_GOES_HERE"

In [ ]:
from openai import OpenAI

HF_TOKEN = "TOKEN_GOES_HERE"

client_olmo = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
)

def query_olmo(prompt):
    completion = client_olmo.chat.completions.create(
        model="allenai/Olmo-3-7B-Instruct:publicai",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=800,
        temperature=0.7,
    )
    return completion.choices[0].message.content


In [ ]:
import pandas as pd
poems_df = pd.read_csv(
    'https://github.com/maria-antoniak/poetry-eval/raw/refs/heads/main/data/poetry-evaluation_public-domain-poems.csv'
)
poems_df = poems_df[poems_df['form'].isin(['sonnet', 'ghazal'])]

sample_df = poems_df[poems_df['form'] == 'sonnet'].sample(25)
sample_df = pd.concat([sample_df, poems_df[poems_df['form'] == 'ghazal']])

In [ ]:
# Load Brown poems
import gdown
url = "https://drive.google.com/file/d/1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz/view?usp=sharing"
output = "brown_poems.csv"
gdown.download(url=url, output=output, fuzzy=True)
brown_df = pd.read_csv("brown_poems.csv")

sample_df = sample_df[['author', 'poem_title', 'poem_text', 'form']]
combined_df_olmo = pd.concat([sample_df, brown_df])

Downloading...
From: https://drive.google.com/uc?id=1SqTw3A99hPDbTUxZ-OoJD4wGdl0ZF3Oz
To: /content/brown_poems.csv
100%|██████████| 3.18k/3.18k [00:00<00:00, 8.71MB/s]


In [ ]:
# Classification template
prompt_template_start = '''
Read the following poem and classify its form as: [sonnet, ghazal, duplex].
You must choose exactly ONE.

Return the answer in this format:

1. Poetic Form: ...
2. Elaborated Rationale: ...
3. One-Word Summary: ...
4. Confidence Score: (0.0-1.0)

Poem Text:
'''

prompt_template_end = '''

Pick ONE of these forms: [sonnet, ghazal, duplex].
'''


In [ ]:
# Classification function
def extract_response_fields(response):
    """Parse Olmo's classification output"""
    lines = response.strip().split('\n')
    result = {
        'poetic_form': None,
        'rationale': None,
        'summary': None,
        'confidence': None
    }

    for line in lines:
        if line.startswith('1. Poetic Form:'):
            result['poetic_form'] = line.split(':', 1)[1].strip()
        elif line.startswith('2. Elaborated Rationale:'):
            result['rationale'] = line.split(':', 1)[1].strip()
        elif line.startswith('3. One-Word Summary:'):
            result['summary'] = line.split(':', 1)[1].strip()
        elif line.startswith('4. Confidence Score:'):
            try:
                result['confidence'] = float(line.split(':', 1)[1].strip())
            except:
                result['confidence'] = 0.0

    return result


In [ ]:
# Process poems
for index, row in combined_df_olmo.iterrows():
    prompt = prompt_template_start + row['poem_text'] + prompt_template_end
    response = query_olmo(prompt)
    fields = extract_response_fields(response)

    combined_df_olmo.at[index, 'poetic_form'] = fields['poetic_form']
    combined_df_olmo.at[index, 'rationale'] = fields['rationale']
    combined_df_olmo.at[index, 'summary'] = fields['summary']
    combined_df_olmo.at[index, 'confidence'] = fields['confidence']

combined_df_olmo.to_csv("combined_df_olmo.csv", index=False)
print(f"Classification complete. Results saved to combined_df_olmo.csv")


Classification complete. Results saved to combined_df_olmo.csv


In [ ]:
# --- Poem Generation with Olmo ---
print("\n--- Starting Olmo Poem Generation ---")

subjects = [
    "abuse", "activities", "arts & sciences", "christianity", "chronic illness",
    "crime & punishment", "cycles", "desire & passion", "doubt & contemplation",
    "family & ancestors", "gender & sexuality", "greek & roman mythology",
    "history & politics", "home life", "hope", "humor & satire",
    "indoor activities", "life choices", "lgbtq+", "living", "love",
    "love death avoidance", "memory & nostalgia", "men & women",
    "mythology & folklore", "nature", "power", "poetry & poets",
    "race & ethnicity", "relationships", "romantic love", "sexual violence",
    "sorrow & grieving", "social commentaries", "subjugation", "the mind",
    "trees and flowers", "truth", "twilight", "urban environment",
    "war & conflict", "weather", "youth"
]


--- Starting Olmo Poem Generation ---


In [ ]:
# Standard prompts
def generate_general_prompt(subject, form="Duplex"):
    return f"Write a poem about the subject of {subject} in the form: {form}."

def generate_figurative_prompt(subject, form="Duplex"):
    return f"Write a poem about {subject} using {form} techniques. Avoid using the words '{subject}' or '{form}'."

def generate_specific_prompt(subject, form="Duplex"):
    return f"Write a specific, concrete poem about {subject} following {form} conventions."


new_poems_df = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])


for subject in subjects:

    genPrompt = generate_general_prompt(subject)
    figPrompt = generate_figurative_prompt(subject)
    specPrompt = generate_specific_prompt(subject)

    genResponse = query_olmo(genPrompt)
    figResponse = query_olmo(figPrompt)
    specResponse = query_olmo(specPrompt)

    # Use concat (append is deprecated)
    rows = [
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'general',   'poem_text': genResponse},
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'figurative','poem_text': figResponse},
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'specific',  'poem_text': specResponse},
    ]

    new_poems_df = pd.concat([new_poems_df, pd.DataFrame(rows)], ignore_index=True)


new_poems_df.to_csv("olmo_general_poems.csv", index=False)
print("Saved olmo_general_poems.csv")


Saved olmo_general_poems.csv


In [ ]:
# Brown rules prompts
BROWN_RULES = """
Follow these strict Duplex rules:
1. Exactly 14 lines
2. 9-11 syllables per line
3. Blend sonnet, ghazal, and blues elements
4. Line 1 = Line 14
5. Line 2 reinterprets Line 1 unexpectedly
6. Line 2 = Line 3
7. Line 4 reinterprets Line 3 unexpectedly
8. Continue echo/reinterpret pattern through Line 13
9. Line 13 starts the final couplet
10. Theme: emotional tension/reflection
"""


In [ ]:
brown_df   = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])

In [ ]:
def generate_brown_general_prompt(subject):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a Duplex poem about the subject '{subject}'. "
        f"Follow ALL Duplex rules exactly."
    )


def generate_brown_figurative_prompt(subject):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a Duplex poem about the subject '{subject}'. "
        f"Follow ALL Duplex rules exactly. "
        f"Do NOT use the words '{subject}' or 'Duplex'."
    )


def generate_brown_specific_prompt(subject):
    return (
        f"{BROWN_RULES}\n\n"
        f"Write a very specific Duplex poem about a concrete moment, object, or memory "
        f"related to '{subject}'. Follow ALL Duplex rules exactly."
    )


brown_poems_df = pd.DataFrame(columns=['subject', 'form', 'prompt_type', 'poem_text'])


# ---- GENERATE POEMS ----
for subject in subjects:

    genPrompt = generate_brown_general_prompt(subject)
    figPrompt = generate_brown_figurative_prompt(subject)
    specPrompt = generate_brown_specific_prompt(subject)

    genResponse = query_olmo(genPrompt)
    figResponse = query_olmo(figPrompt)
    specResponse = query_olmo(specPrompt)

    # collect rows
    rows = [
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'general',   'poem_text': genResponse},
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'figurative','poem_text': figResponse},
        {'subject': subject, 'form': 'Duplex', 'prompt_type': 'specific',  'poem_text': specResponse},
    ]


    brown_poems_df = pd.concat([brown_poems_df, pd.DataFrame(rows)], ignore_index=True)


# ---- SAVE FILE ----
brown_poems_df.to_csv("olmo_brown_poems.csv", index=False)

print("\nPoem generation complete. Files saved:")
print("  - olmo_general_poems.csv")
print("  - olmo_brown_poems.csv")


Poem generation complete. Files saved:
  - olmo_general_poems.csv
  - olmo_brown_poems.csv
